## 1. Install the theos python package

In [1]:
!sudo apt install tesseract-ocr && wget https://bootstrap.pypa.io/get-pip.py && python3.9 get-pip.py && python3.9 -m pip install --upgrade pip && python3.9 -m pip install theos-ai[ocr]==0.0.28

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 22 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 0s (29.8 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fron

## 2. Run the setup to finish installing theos

In [2]:
!theos setup --version v1 --subfield object-detection

[theos] setting up object-detection subfield...

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.1/283.1 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 90.4 MB/s 

## 3. Restart the colab runtime

Click the restart runtime option on the top menu.

<img src="https://drive.google.com/uc?id=127ow4FbCF8cfBVfC-bWdhsv4l0-Usb2m" width="650px"/>


## 4. Log in to your account

In [1]:
!theos login


 Theos CLI

[*] username: rahulkarmakar
[*] password: 

[theos] login succeeded


# 5. Set your theos project
Replace **\<project_key>** with your theos project key.


In [2]:
!theos set-project 252539db-5f84-4228-96cd-bb8fc0af6a76

[theos] project added successfully


# 6. Install the algorithm and weights
Replace the following parameters with your own values:


*   **\<algorithm>**: the algorithm name, such as **yolov7**
*   **\<algorithm_version>**: the algorithm version, such as **tiny**
*   **\<weights_tag>**: the weights tag, such as **license-plates:experiment-1:best**



In [3]:
!theos install yolov7 --version tiny --weights license-plate-detection:experiment-2:best

[theos] downloading yolov7 code:

100% 272k/272k [00:00<00:00, 659kB/s]

[theos] extracting yolov7 code...
[theos] downloading license-plate-detection weights for the tiny version of yolov7:

100% 11.7M/11.7M [00:01<00:00, 7.34MB/s]

[theos] license-plate-detection experiment-2:best weights downloaded successfully ✓


# 7. Detect and read
Replace configuration variables with your own values.

In [6]:
ALGORITHM = 'yolov7' # set the algorithm, such as 'yolov7'
ALGORITHM_VERSION = 'tiny' # set the algorithm version, such as 'tiny'
WEIGHTS = 'license-plate-detection:experiment-2:best' # set the weights tag, such as 'license-plates:experiment-1:best'

OCR_MODEL_SIZE = 'large' # set the OCR model size, possible values are 'small', 'medium' and 'large'
OCR_MODEL_TYPE = 'printed' # set the OCR model type if using OCR_MODEL_SIZE = 'large', possible values are 'str', 'printed' and 'handwritten'
OCR_MODEL_ACCURACY = 'best' # set the OCR model accuracy if using OCR_MODEL_SIZE = 'large', possible values are 'base', 'medium' and 'best'
OCR_CLASSES = ['license_plate'] # set the class names that the OCR model should read from, such as ['license-plate']

INPUT_VIDEO = 'video.mp4'
OUTPUT_VIDEO = 'output.mp4'

Drop the video to the colab instance filesystem, rename it to the **INPUT_VIDEO** filename, and run the following detection and OCR script.

This script will detect and perform OCR on your video and save the output video to your machine.

In [7]:
from theos.computer_vision.object_detection.utils import draw
from theos.computer_vision import ocr
from theos.client import Client
from google.colab import files
from tqdm import tqdm
import torch
import json
import time
import cv2
import os

if torch.cuda.is_available():
  torch.cuda.empty_cache()

client = Client(inputs='.', outputs='.')
yolov7 = client.get(ALGORITHM, version=ALGORITHM_VERSION)
yolov7.load_weights(WEIGHTS)
yolov7.to_gpu()

if OCR_MODEL_TYPE and OCR_MODEL_ACCURACY: 
  ocr_model = ocr.load(size=OCR_MODEL_SIZE, model_type=OCR_MODEL_TYPE, accuracy=OCR_MODEL_ACCURACY)
else:
  ocr_model = ocr.load(size=OCR_MODEL_SIZE)

if OCR_MODEL_SIZE == 'large':
  ocr_model.to_gpu()

video = cv2.VideoCapture(INPUT_VIDEO)
width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
frames_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

if video.isOpened() == False:
	print('[!] error opening the video')

print('[theos] detecting video...\n')
pbar = tqdm(total=frames_count, unit=' frames', dynamic_ncols=True, position=0, leave=True)

while video.isOpened():
    ret, frame = video.read()
    if ret == True:
        detections = yolov7.detect(frame)
        detections = ocr_model.read(frame, detections=detections, classes=OCR_CLASSES)
        detected_frame = draw(frame, detections, alpha=0.15)
        output.write(detected_frame)
        pbar.update(1)
    else:
        break

pbar.close()
print('\n')
video.release()
output.release()
yolov7.unload()
client.close()

print('[theos] downloading output video...\n')
files.download(OUTPUT_VIDEO)

[theos] importing yolov7 code...
[theos] yolov7 code imported successfully
[theos] loading yolov7...
[theos] loading the large OCR model...
[theos] large OCR model loaded successfully
[theos] detecting video...



100%|██████████| 527/527 [00:09<00:00, 53.88 frames/s]




[theos] unloading instance of yolov7...
[theos] done
[theos] downloading output video...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>